In [1]:
from shapely import wkt
from shapely.geometry import Point
import geojson
import pandas as pd
import numpy as np
import keplergl
from odps.df import DataFrame
from util import GisTransform
import json
from shapely.geometry import MultiPoint, Polygon, MultiPolygon, Point
import geopandas
import hashlib

gis = GisTransform('wgs84', 'gcj02')

In [2]:
partial = pd.read_csv('spider.csv')
df = pd.read_csv('temp_dw_ai_normal_deliver_range_aoi_basic_chengdu_20200929_1601451326255.txt')

In [3]:
partial

,id,geohash,is_dispatch
0,B001C8VMWD,wm3vy8,1.0
1,B0FFJ7UHWY,wm3vy8,1.0
2,B001C7UWK4,wm3vy8,1.0
3,B001C8X2IQ,wm3vy8,1.0
4,B0FFGKP5IZ,wm3vy8,1.0
...,...,...,...
10154,B001C7WRXZ,wm3ywg,1.0
10155,B001C80QAX,wm3ywg,1.0
10156,B001C8081D,wm3ywg,1.0
10157,B001C80FS2,wm3ywg,1.0


In [4]:
m = pd.merge(partial, df[['id', 'shape', 'name']], on='id', how='inner')

In [12]:
pd.merge(partial, df[['id', 'name']], on='id', how='inner').to_csv('report.csv', index=False)

In [5]:
def transfer_to_geojson(poly):
    if isinstance(poly, float):
        return None
    
    points = []
    for item in poly.split(';'):
        arr = item.split(',')
        pair = gis.gcj02_to_wgs84(float(arr[0]), float(arr[1]))
        points.append([pair[0], pair[1]])

    polygon = {
      "type": "Feature",
      "geometry": {
        "type": "Polygon",
        "coordinates": [points]
      },
      "properties": {
        "name": "shop delivery area"
      }
    }
    
    return polygon

In [6]:
m['geojson'] = m['shape'].apply(lambda item: transfer_to_geojson(item))

In [7]:
m[m['is_dispatch']==1].iloc[0]['geojson']

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[104.00081192359815, 30.546598653148358],
    [104.00060766280795, 30.546955157172427],
    [104.00004774178247, 30.546738933624667],
    [104.00024901477877, 30.54634944924044],
    [104.00081192359815, 30.546598653148358]]]},
 'properties': {'name': 'shop delivery area'}}

In [8]:
amap = keplergl.KeplerGl(height=800)  
amap.add_data(data=m[m['is_dispatch']==1], name='cover region')
amap

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'cover region': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19…

In [10]:
amap.save_to_html()

Map saved to keplergl_map.html!
